In [ ]:
import numpy as np
import os
import shutil
import tensorflow as tf
from tensorflow import keras
from PIL import Image

In [ ]:
cd '/content/gdrive/My Drive/TensorFlow_Training_13th'

In [ ]:
cur_dir = os.getcwd()

train_dir = os.path.join(cur_dir, 'simpsons_train')
val_dir = os.path.join(cur_dir, 'simpsons_val')

n_class = len(os.listdir(train_dir))
print("This Dataset has {} classes".format(n_class))

n_train = 0
n_val = 0
for _, _, files in os.walk(train_dir):
    if files:
        n_train += len(files)
for _, _, files in os.walk(val_dir):
    if files:
        n_val += len(files)
        
print("Train Dataset has {} images".format(n_train))
print("Validation Dataset has {} images".format(n_val))

In [ ]:
img_size = 150

In [ ]:
tfrecord_dir = os.path.join(cur_dir, 'tfrecords')
tfrecord_train = 'simpsons_train.tfrecord'
tfrecord_val = 'simpsons_val.tfrecord'

os.makedirs(tfrecord_dir, exist_ok=True)

In [ ]:
train_tfr_dir = os.path.join(tfrecord_dir, tfrecord_train)
val_tfr_dir = os.path.join(tfrecord_dir, tfrecord_val)

writer_train = tf.io.TFRecordWriter(train_tfr_dir)
writer_val = tf.io.TFRecordWriter(val_tfr_dir)

In [ ]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [ ]:
train_classes = sorted([dname for dname in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, dname))])
n_train = 0
for label, train_class in enumerate(train_classes):
    print("saving data from {}...".format(train_class))
    cat_dir = os.path.join(train_dir, train_class)
    fnames = os.listdir(cat_dir)
    
    for fname in fnames:
        fpath = os.path.join(cat_dir, fname)
        img = Image.open(fpath)
        img = img.resize((img_size, img_size))
        img = np.asarray(img)          
        img_str = img.tobytes()
        example = tf.train.Example(features=tf.train.Features(feature={
            'image': _bytes_feature(img_str),
            'label': _int64_feature(label)}))
        writer_train.write(example.SerializeToString())
        n_train += 1
print("{} train images are saved".format(n_train))

In [ ]:
val_classes = sorted([dname for dname in os.listdir(val_dir) if os.path.isdir(os.path.join(val_dir, dname))])
n_val = 0
for label, val_class in enumerate(val_classes):
    print("saving data from {}...".format(val_class))
    cat_dir = os.path.join(val_dir, val_class)
    fnames = os.listdir(cat_dir)
    
    for fname in fnames:
        fpath = os.path.join(cat_dir, fname)
        img = Image.open(fpath)
        img = img.resize((img_size, img_size))
        img = np.asarray(img)
        img_str = img.tobytes()
        example = tf.train.Example(features=tf.train.Features(feature={
            'image': _bytes_feature(img_str),
            'label': _int64_feature(label)}))
        writer_val.write(example.SerializeToString())
        n_val += 1
print("{} validation images are saved".format(n_val))